__This notebook__ explains how to fine-tune GPT-2 Large on a large dataset in colab or on your home computer.

To fit this task into a colab instance, we will use two tricks:
* streaming the [C4 dataset](https://huggingface.co/datasets/c4) using [`datasets` Streaming API](https://huggingface.co/docs/datasets/dataset_streaming.html). Without that, C4 would take up over 300GB of disk space.
* training with 8-Bit Adam from the [`bitsandbytes` library](https://github.com/facebookresearch/bitsandbytes). Without 8-bit compression, training GPT-2 Large would not fit in GPU memory.



This notebook is based on the ["fine-tune a language model"](https://github.com/huggingface/notebooks/blob/master/examples/language_modeling.ipynb) tutorial by [Sylvain Gugger](https://sgugger.github.io/pages/about-me.html#about-me) as well as the [pytorch language-model example](https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_clm_no_trainer.py).

# Installation

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [40]:
 ! pip install datasets transformers


We are also installing bitsandbytes which depends on the CUDA version run by your colab. The installed CUDA version is displayed in the top right when calling nvidia-smi. Use this version to install the right bitsandbytes version below. We need a GPU for this, so if you have not yet a GPU loaded use Runtime-> Change runtime type -> select GPU from the dropdown.

In [41]:
! nvidia-smi
! pip install bitsandbytes-cuda112

Thu Apr  7 17:04:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    32W /  70W |   1324MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

To test the bitsandbytes installation we can run a simple update with 8-bit Adam.

In [42]:
import bitsandbytes as bnb
import torch

p = torch.nn.Parameter(torch.rand(10,10).cuda())
a = torch.rand(10,10).cuda()

p1 = p.data.sum().item()

adam = bnb.optim.Adam8bit([p])

out = a*p
loss = out.sum()
loss.backward()
adam.step()

p2 = p.data.sum().item()

assert p1 != p2
print('SUCCESS!')
print('Installation was successful!')

SUCCESS!
Installation was successful!


# Dataset Streaming

Pre-training often requires a huge text dataset. Some famous datasets for pre-training are [C4](https://huggingface.co/datasets/c4), its multilingual version [mC4](https://huggingface.co/datasets/mc4), as well as [OSCAR](https://huggingface.co/datasets/oscar).

These datasets can be terabytes of data and require a lot of resources:
- a good bandwidth to download all the data
- terabytes of disk space to store the data
- dozens of CPUs and a good infrastructure to tokenize the text dataset
- lots of time to wait for the tokenization to happen

This makes it very impractical to get your hands on a dataset for pretraining when you have limited resources.

Dataset streaming is the solution in this case. Streaming allows to simply have access to the very small subset of the dataset that you need at any time during training. Text samples are progressively downloaded during training, and processed on-the-fly.

Thanks to dataset streaming:
- training can start directly without waiting for terabytes of data to be downloaded
- you can use an arbitrarily large dataset, without being constrained by your disk space
- you can process the batches of text as they arrive with a regular CPU
- you don't waste time processing examples that are not immediately needed for training

Dataset streaming is available in [Hugging Face Datasets](https://github.com/huggingface/datasets) - you simply need to pass `streaming=True` when loading a dataset, and it can be used with a PyTorch data loader.

![dataset streaming](https://huggingface.co/docs/datasets/_images/stream.gif "Dataset Streaming")

Hugging face Datasets also allows you to process examples on-the-fly via `.map()` and shuffle the dataset with `.shuffle()`.

Here is an example on how to load the C4 dataset in streaming mode:

In [2]:
# Load DS for finetuning https://www.kaggle.com/code/tanmay111/youtube-comments-sentiment-analysis/data
import pandas as pd

US_comments = pd.read_csv('data/UScomments.csv', error_bad_lines=False)
US_videos = pd.read_csv('data/USvideos.csv', error_bad_lines=False)

b'Skipping line 41589: expected 4 fields, saw 11\nSkipping line 51628: expected 4 fields, saw 7\nSkipping line 114465: expected 4 fields, saw 5\n'
b'Skipping line 142496: expected 4 fields, saw 8\nSkipping line 189732: expected 4 fields, saw 6\nSkipping line 245218: expected 4 fields, saw 7\n'
b'Skipping line 388430: expected 4 fields, saw 5\n'
b'Skipping line 2401: expected 11 fields, saw 21\nSkipping line 2800: expected 11 fields, saw 21\nSkipping line 5297: expected 11 fields, saw 12\nSkipping line 5299: expected 11 fields, saw 12\nSkipping line 5300: expected 11 fields, saw 12\nSkipping line 5301: expected 11 fields, saw 12\n'


In [69]:
# Let's do some analysis and Data Cleaning on both the datasets
print(f"US_videos.shape = {US_videos.shape}")
print(f"US_videos.columns = {US_videos.columns}")
print(f"US_comments.shape = {US_comments.shape}")
print(f"US_comments.columns = {US_comments.columns}")

US_comments.drop(41587, inplace=True)
US_comments = US_comments.reset_index().drop('index',axis=1)
US_comments.likes = US_comments.likes.astype(int)
US_comments.replies = US_comments.replies.astype(int)
US_comments.comment_text = US_comments.comment_text.astype(str)

# Removing Punctuations, Numbers and Special Characters.
US_comments['comment_text'] = US_comments['comment_text'].str.replace("[^a-zA-Z#]", " ")
# # Removing Short Words.
# US_comments['comment_text'] = US_comments['comment_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# Changing the text to lower case
US_comments['comment_text'] = US_comments['comment_text'].apply(lambda x:x.lower())

# Tokenization
tokenized_tweet = US_comments['comment_text'].apply(lambda x: x.split())
# Lemmatization
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords

# wnl = WordNetLemmatizer()
# tokenized_tweet.apply(lambda x: [wnl.lemmatize(i) for i in x if i not in set(stopwords.words('english'))]) 
# tokenized_tweet.head()
# for i in range(len(tokenized_tweet)):
#     tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
# US_comments['comment_text'] = tokenized_tweet

US_videos.shape = (7992, 11)
US_videos.columns = Index(['video_id', 'title', 'channel_title', 'category_id', 'tags', 'views',
       'likes', 'dislikes', 'comment_total', 'thumbnail_link', 'date'],
      dtype='object')
US_comments.shape = (691397, 4)
US_comments.columns = Index(['video_id', 'comment_text', 'likes', 'replies'], dtype='object')


0                                        [logan, paul]
1    [been, following, from, start, your, vine, cha...
2                                     [kong, maverick]
3                                         [attendance]
4                                           [trending]
Name: comment_text, dtype: object

In [78]:
from datasets import load_dataset

c4 = load_dataset("c4", "en", streaming=True)
# # Let's print a few examples
for i, example in enumerate(c4["train"]):
  print(example.keys())
    # print(f"{i}: {str(example)[:200]}...")
    # if i == 5:
    #     break


Выходные данные были обрезаны до нескольких последних строк (5000).
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['text', 'timestamp', 'url'])
dict_keys(['

KeyboardInterrupt: ignored

# Fine-tuning a language model

We are fine-tuning without Trainer so we can specify dataset streaming and 8-bit optimizers. For that, we copy the main parts of the [example script for language modeling](https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_clm_no_trainer.py). First we import boiler-plate and the go step-by-step throught the model setup and training.

## Preprocessing & model setup

In [ ]:
# You can also adapt this script on your own causal language modeling task. Pointers for this are left as comments.

import argparse
import logging
import math
import os
import random
from itertools import chain
from pathlib import Path

import datasets
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import transformers
from huggingface_hub import Repository
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)
from transformers.file_utils import get_full_repo_name
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)

require_version("datasets>=1.16.1", "To fix: pip install -r examples/pytorch/language-modeling/requirements.txt")

MODEL_CONFIG_CLASSES = list(MODEL_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)


def parse_args():
    parser = argparse.ArgumentParser(description="Finetune a transformers model on a causal language modeling task")
    parser.add_argument(
        "--dataset_name",
        type=str,
        default=None,
        help="The name of the dataset to use (via the datasets library).",
    )
    parser.add_argument(
        "--dataset_config_name",
        type=str,
        default=None,
        help="The configuration name of the dataset to use (via the datasets library).",
    )
    parser.add_argument(
        "--text_column_name",
        type=str,
        default=None,
        help="The name of the column containing the text data.",
    )
    parser.add_argument(
        "--dataset_streaming",
        action="store_true",
        help="If passed, will use dataset streaming (via the datasets library)",
    )
    parser.add_argument(
        "--model_name_or_path",
        type=str,
        help="Path to pretrained model or model identifier from huggingface.co/models.",
        required=False,
    )
    parser.add_argument(
        "--config_name",
        type=str,
        default=None,
        help="Pretrained config name or path if not the same as model_name",
    )
    parser.add_argument(
        "--tokenizer_name",
        type=str,
        default=None,
        help="Pretrained tokenizer name or path if not the same as model_name",
    )
    parser.add_argument(
        "--use_slow_tokenizer",
        action="store_true",
        help="If passed, will use a slow tokenizer (not backed by the 🤗 Tokenizers library).",
    )
    parser.add_argument(
        "--per_device_train_batch_size",
        type=int,
        default=1,
        help="Batch size (per device) for the training dataloader.",
    )
    parser.add_argument(
        "--learning_rate",
        type=float,
        default=5e-5,
        help="Initial learning rate (after the potential warmup period) to use.",
    )
    parser.add_argument("--weight_decay", type=float, default=0.0, help="Weight decay to use.")
    parser.add_argument("--num_train_epochs", type=int, default=1, help="Total number of training epochs to perform.")
    parser.add_argument(
        "--max_train_steps",
        type=int,
        default=None,
        help="Total number of training steps to perform. If provided, overrides num_train_epochs.",
    )
    parser.add_argument(
        "--gradient_accumulation_steps",
        type=int,
        default=1,
        help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument(
        "--lr_scheduler_type",
        type=SchedulerType,
        default="linear",
        help="The scheduler type to use.",
        choices=["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"],
    )
    parser.add_argument(
        "--num_warmup_steps", type=int, default=3000, help="Number of steps for the warmup in the lr scheduler."
    )
    parser.add_argument("--output_dir", type=str, default=None, help="Where to store the final model.")
    parser.add_argument("--seed", type=int, default=None, help="A seed for reproducible training.")
    parser.add_argument(
        "--model_type",
        type=str,
        default=None,
        help="Model type to use if training from scratch.",
        choices=MODEL_TYPES,
    )
    parser.add_argument(
        "--block_size",
        type=int,
        default=None,
        help="Optional input sequence length after tokenization. The training dataset will be truncated in block of this size for training. Default to the model max input length for single sentence inputs (take into account special tokens).",
    )
    parser.add_argument(
        "--preprocessing_num_workers",
        type=int,
        default=None,
        help="The number of processes to use for the preprocessing.",
    )
    parser.add_argument(
        "--overwrite_cache", type=bool, default=False, help="Overwrite the cached training and evaluation sets"
    )
    parser.add_argument(
        "--no_keep_linebreaks", action="store_true", help="Do not keep line breaks when using TXT files."
    )
    parser.add_argument("--push_to_hub", action="store_true", help="Whether or not to push the model to the Hub.")
    parser.add_argument(
        "--hub_model_id", type=str, help="The name of the repository to keep in sync with the local `output_dir`."
    )
    parser.add_argument("--hub_token", type=str, help="The token to use to push to the Model Hub.")
    args = parser.parse_args(args=[])

    if args.push_to_hub:
        assert args.output_dir is not None, "Need an `output_dir` to create a repo when `--push_to_hub` is passed."

    return args


We setup the streaming dataset, the tokenizer, and the model (GPT-2 medium).

In [ ]:
args = parse_args() # get default arguments

# If passed along, set the training seed now.
if args.seed is not None:
    set_seed(args.seed)

args.dataset_name = 'c4'
args.dataset_streaming = True
args.text_column_name = "text"
args.model_name_or_path = 'gpt2-large'
args.dataset_config_name = "en"
args.block_size = 1024
args.max_train_steps = 1_000_000
args.log_loss_interval = 25


# LOAD DATA
raw_train_dataset = load_dataset(args.dataset_name, args.dataset_config_name, streaming=args.dataset_streaming, split="train")

if args.config_name:
    config = AutoConfig.from_pretrained(args.config_name)
elif args.model_name_or_path:
    config = AutoConfig.from_pretrained(args.model_name_or_path)
else:
    config = CONFIG_MAPPING[args.model_type]()
    logger.warning("You are instantiating a new config instance from scratch.")

# TOKENIZER
if args.tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=not args.use_slow_tokenizer)
elif args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=not args.use_slow_tokenizer)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )

if args.model_name_or_path:
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
    )
else:
    logger.info("Training new model from scratch")
    model = AutoModelForCausalLM.from_config(config)

model.resize_token_embeddings(len(tokenizer))

model.gradient_checkpointing_enable()
model.cuda() # send model to cuda preemptively to free RAM. Yep, we're using GPU as an offload memory. O tempora! O mores!

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02G [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

We now preprocess the dataset into blocks of the specified size (768).

In [ ]:
text_column_name = args.text_column_name

def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_train_dataset = raw_train_dataset.shuffle(10_000, seed=42).map(tokenize_function, batched=True)

if args.block_size is None:
    block_size = tokenizer.model_max_length
    if block_size > 1024:
        logger.warning(
            f"The tokenizer picked seems to have a very large `model_max_length` ({tokenizer.model_max_length}). "
            "Picking 1024 instead. You can change that default value by passing --block_size xxx."
        )
    block_size = 1024
else:
    if args.block_size > tokenizer.model_max_length:
        logger.warning(
            f"The block_size passed ({args.block_size}) is larger than the maximum length for the model"
            f"({tokenizer.model_max_length}). Using block_size={tokenizer.model_max_length}."
        )
    block_size = min(args.block_size, tokenizer.model_max_length)

# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a remainder
# for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value might be slower
# to preprocess.
train_dataset = tokenized_train_dataset.map(group_texts, batched=True)
train_dataset = train_dataset.shuffle(10_000, seed=42).with_format("torch")

# DataLoaders creation:
train_dataloader = DataLoader(
    train_dataset, collate_fn=default_data_collator, batch_size=args.per_device_train_batch_size
)

## 8-bit Optimizers


In this example, we fine-tune GPT-2 medium with a sequence dimension of 768 which runs out of memory. How can we fit this model on a colab GPU with 12 GB of memory? One solution is to use 8-bit optimizers.

8-bit opitimizers decrease the memory footprint for training models by compressing and storing the optimizer statistics for optimizers. For Adam, there are two optimizer buffers, one for an estimate of the running mean of the gradient and one for the standard deviation. Each of the buffers has the size of the full model, as such, the Adam optimizers uses 2x more memory than the model itself. With 8-bit optimizers we reduce that from 32-bit to 8-bit thus reducing the memory due to Adam from 2x the model size to 0.5x the model size -- a reduction by 75%.

8-bit optimizers work by using dynamic quantization and block-wise quantization to ensure stable training and the same performance as 32-bit optimizers while achieving the 75% reduction in memory.

8-bit optimizers work as follows
1. Chunk optimizer states into blocks
2. Normalize each block into the range [-1, 1] by dividing by the absmax of the block
3. Perform dynamic quantization
4. Store 8-bit data

For dequantization we reverse these steps. These steps are demonstrated by the example below:

 ![Schematic of 8-bit optimizers](https://timdettmers.com/wp-content/uploads/2021/12/schematic2.svg)



In [ ]:
import bitsandbytes as bnb
#optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate) # this crashes with out-of-memory error
optimizer = bnb.optim.Adam8bit(model.parameters(), lr=args.learning_rate)

## Train the model

Now we are training the model with dataset streaming and 8-bit optimizers.

In [ ]:
lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps,
    num_training_steps=args.max_train_steps,
)
# Train!
total_batch_size = args.per_device_train_batch_size * args.gradient_accumulation_steps
# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), disable=False)
completed_steps = 0

def get_free_mem():
    t = torch.cuda.get_device_properties(0).total_memory
    r = torch.cuda.memory_reserved(0)
    a = torch.cuda.memory_allocated(0)
    f = r - a
    return f/1024**3, r/1024**3, a/1024**3

for epoch in range(args.num_train_epochs):
    model.train()
    losses = []
    for step, batch in enumerate(train_dataloader):
        gpu_data = {}
        for key, value in batch.items():
            gpu_data[key] = value.cuda()

        outputs = model(**gpu_data, use_cache=False)
        loss = outputs.loss
        losses.append(loss.item())
        loss = loss / args.gradient_accumulation_steps
        loss.backward()
        if step % args.gradient_accumulation_steps == 0 or step == args.max_train_steps:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            completed_steps += 1

        if step % args.log_loss_interval == 0 and step > 0:
            try:
                perplexity = math.exp(sum(losses)/len(losses))
            except OverflowError:
                perplexity = float("inf")
            losses = []
            print(f"epoch: {epoch+1}, step: {step}, perplexity: {perplexity}")
            

        if completed_steps >= args.max_train_steps:
            break



  0%|          | 0/1000000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (11128 > 1024). Running this sequence through the model will result in indexing errors


epoch: 1, step: 25, perplexity: 21.84923336079448


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-5aee53072d76>", line 32, in <module>
    loss.backward()
  File "/usr/local/lib/python3.7/dist-packages/torch/_tensor.py", line 307, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py", line 156, in backward
    allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the 

KeyboardInterrupt: ignored